In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
from sklearn.datasets import *
import pandas as pd
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

/home/grads/aafiyahussain/miniconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-19 21:29:00,486	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
sampling_params = SamplingParams(max_tokens=512,temperature=1.2)

In [4]:
model_id = "microsoft/Phi-4-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
llm = LLM(
    model=model_id,
    tensor_parallel_size=1,
    max_model_len=8192
)

INFO 04-19 21:29:09 __init__.py:207] Automatically detected platform cuda.
INFO 04-19 21:29:10 config.py:208] Replacing legacy 'type' key with 'rope_type'
INFO 04-19 21:29:20 config.py:549] This model supports multiple tasks: {'embed', 'reward', 'classify', 'score', 'generate'}. Defaulting to 'generate'.
INFO 04-19 21:29:20 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='microsoft/Phi-4-mini-instruct', speculative_config=None, tokenizer='microsoft/Phi-4-mini-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:23<00:23, 23.85s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [01:06<00:00, 34.73s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [01:06<00:00, 33.09s/it]



INFO 04-19 21:32:39 model_runner.py:1115] Loading model weights took 7.1694 GB
INFO 04-19 21:32:40 worker.py:267] Memory profiling takes 1.36 seconds
INFO 04-19 21:32:40 worker.py:267] the current vLLM instance can use total_gpu_memory (44.43GiB) x gpu_memory_utilization (0.90) = 39.99GiB
INFO 04-19 21:32:40 worker.py:267] model weights take 7.17GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.87GiB; the rest of the memory reserved for KV Cache is 30.89GiB.
INFO 04-19 21:32:40 executor_base.py:111] # cuda blocks: 15817, # CPU blocks: 2048
INFO 04-19 21:32:40 executor_base.py:116] Maximum concurrency for 8192 tokens per request: 30.89x
INFO 04-19 21:32:42 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_ut

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:14<00:00,  2.46it/s]

INFO 04-19 21:32:57 model_runner.py:1562] Graph capturing finished in 14 secs, took 0.41 GiB
INFO 04-19 21:32:57 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 17.95 seconds


In [5]:
def get_prompt(summary_statistics,sampled_rows):
    prompt = f"""You are a data visualization expert whose goal is to suggest most insighful visualization from a list of options.  
    You will be provided with two inputs:

    1. **Summary statistics** (the comma seperated output of `df.describe()`)  
    2. **Ten sample rows** from the dataset, each row on its own line, with comma‑separated values.

    Your task is to:

    1. Choose **one** visualization from this list of common exploratory plots:
    - Histogram  
    - Box plot  
    - Scatter plot  
    - Bar chart  
    - Line chart  
    - Heatmap (e.g. correlation matrix)  
    - Pair plot (scatterplot matrix)  
    - Violin plot  
    - Density plot  
    - Stacked bar chart  

    2. Explain your **rationale** for selecting that visualization given the summary statistics and sample rows.

    Inputs:
    1. Summary statistics:
    {summary_statistics}

    2. Ten randomly selected rows from the dataset:
    {sampled_rows}

    Provide your answer in the format \\boxed{{answer}} at the end."""
    return prompt

In [6]:
def get_summary_stat_and_rows(df):
    summary_statistics = df.describe().to_csv(index=True)
    sampled_rows = df.sample(n=10, random_state=42).to_csv(index=False)
    return summary_statistics,sampled_rows

## Iris

In [7]:
iris = load_iris()
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris['target'] = [iris.target_names[t] for t in iris.target]
# csv_data = df_iris.to_csv(index=False)

In [8]:
summary_statistics, sampled_rows = get_summary_stat_and_rows(df_iris)

In [9]:
prompt = get_prompt(summary_statistics,sampled_rows)

In [10]:
messages = [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

In [11]:
outputs = llm.generate(prompts, sampling_params, use_tqdm=False)
outputs

[RequestOutput(request_id=0, prompt='<|user|>You are a data visualization expert whose goal is to suggest most insighful visualization from a list of options.  \n    You will be provided with two inputs:\n\n    1. **Summary statistics** (the comma seperated output of `df.describe()`)  \n    2. **Ten sample rows** from the dataset, each row on its own line, with comma‑separated values.\n\n    Your task is to:\n\n    1. Choose **one** visualization from this list of common exploratory plots:\n    - Histogram  \n    - Box plot  \n    - Scatter plot  \n    - Bar chart  \n    - Line chart  \n    - Heatmap (e.g. correlation matrix)  \n    - Pair plot (scatterplot matrix)  \n    - Violin plot  \n    - Density plot  \n    - Stacked bar chart  \n\n    2. Explain your **rationale** for selecting that visualization given the summary statistics and sample rows.\n\n    Inputs:\n    1. Summary statistics:\n    ,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)\ncount,150.0,150.0,

In [12]:
outputs[0].outputs[0].text #scatter plot

"Given the provided summary statistics and sample rows, I would choose the **scatter plot** as the most insightful visualization.\n\n**Rationale:**\n- **Summary Statistics:** The provided statistics mostly cover the central tendency and spread of the features. However, to see distributions, relationships, and potential patterns, we need to visualize these declines together.\n- **Sample Rows:** Given the species combinations from the blooming datasets typical of the Iris dataset, we can identify how the sepal and petal dimensions differentiate between the various species types (e.g., setosa, versicolor, virginica). Scatter plots effectively exhibit the relationships and patterns between pairwise combinations of variables, thereby highlighting how one variable might influence another and distinguishing the peaks corresponding to different species.\n\nFor instance, with the quoted pairs replicated:\n- Scatter plot axes could illustrate; petal length vs. sepal length with different markers

In [1]:
s = "Given the provided summary statistics and sample rows, I would choose the **scatter plot** as the most insightful visualization.\n\n**Rationale:**\n- **Summary Statistics:** The provided statistics mostly cover the central tendency and spread of the features. However, to see distributions, relationships, and potential patterns, we need to visualize these declines together.\n- **Sample Rows:** Given the species combinations from the blooming datasets typical of the Iris dataset, we can identify how the sepal and petal dimensions differentiate between the various species types (e.g., setosa, versicolor, virginica). Scatter plots effectively exhibit the relationships and patterns between pairwise combinations of variables, thereby highlighting how one variable might influence another and distinguishing the peaks corresponding to different species.\n\nFor instance, with the quoted pairs replicated:\n- Scatter plot axes could illustrate; petal length vs. sepal length with different markers for different the colors specific to labels in the target variable, precisely demonstrating their distributions and differentially signifying distinct clusters.\n\nThis will allow meaningful insights regarding the distinguishing species' measured variances over respective features contributing to the differentiated species peak platformalinguistics in echo with Garden types or similarly laying invoking incidence improvizzability to occe and observations translations effectively between pairs' predicted generativity.\n\nIn summary:\n\n\\boxed{scatter plot}"
print(s)

Given the provided summary statistics and sample rows, I would choose the **scatter plot** as the most insightful visualization.

**Rationale:**
- **Summary Statistics:** The provided statistics mostly cover the central tendency and spread of the features. However, to see distributions, relationships, and potential patterns, we need to visualize these declines together.
- **Sample Rows:** Given the species combinations from the blooming datasets typical of the Iris dataset, we can identify how the sepal and petal dimensions differentiate between the various species types (e.g., setosa, versicolor, virginica). Scatter plots effectively exhibit the relationships and patterns between pairwise combinations of variables, thereby highlighting how one variable might influence another and distinguishing the peaks corresponding to different species.

For instance, with the quoted pairs replicated:
- Scatter plot axes could illustrate; petal length vs. sepal length with different markers for dif

## Diabetes

In [13]:
diabetes = load_diabetes()
df_diabetes = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)

In [14]:
summary_statistics,sampled_rows = get_summary_stat_and_rows(df_diabetes)
prompt = get_prompt(summary_statistics,sampled_rows)

In [15]:
messages = [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

In [16]:
outputs = llm.generate(prompts, sampling_params, use_tqdm=False)
outputs[0].outputs[0].text #Histogram

"\\boxed{Histogram}\n\n**Rationale:** \n\nThe given summary statistics provide insights predominantly within the first six continuous variables: age, sex, BMI, BP, and four additional features (s1 to s6). Observing the mean, typical values, and variations captured by the standard deviation, we see that the variables cover both types that like normally distributed data as well as uniformly distributed ranges. Given this, a histogram would significantly benefit in visualizing and identifying the shape, spread, and central tendencies of these continuous values, noticing the identifiable peaks and spread (which can especially help with variables like age which can have skewness and multimodality in a larger dataset). Using plots as diversely as the others won't depict that specifically. Hence: \n\n1. Varied spread and outliers - histograms effectively show smooth distributions and variability, such as age-age who share nearly the same age or BMI following normal (equal spread).\n2. The gro

In [2]:
s = "\\boxed{Histogram}\n\n**Rationale:** \n\nThe given summary statistics provide insights predominantly within the first six continuous variables: age, sex, BMI, BP, and four additional features (s1 to s6). Observing the mean, typical values, and variations captured by the standard deviation, we see that the variables cover both types that like normally distributed data as well as uniformly distributed ranges. Given this, a histogram would significantly benefit in visualizing and identifying the shape, spread, and central tendencies of these continuous values, noticing the identifiable peaks and spread (which can especially help with variables like age which can have skewness and multimodality in a larger dataset). Using plots as diversely as the others won't depict that specifically. Hence: \n\n1. Varied spread and outliers - histograms effectively show smooth distributions and variability, such as age-age who share nearly the same age or BMI following normal (equal spread).\n2. The growing frequency gutters ensure an elongated visual for that particular patching which probably won't be possible in linear bar showcasing S/N comparison for just one variable each. \n3. Most conclusions straight and diss; easily on utilizing the paired arrangements otherwise like violin plots multicorrelations aren't outlined reducible and not overall pulling inefficiencies. Aggregating all histogram sets could greatly help domain expert use visual shorthand further reassurance another set of viewpoint differentiation sake.\n\nAdditionally, visualizing in multiple groups would gust effectively into histograms facilitate seamless observation of bar-stacking unionally otherwise emphasized steps seldom catered smoothly and intuitive throughout their jack to streamline decision mapping or evocative comparable scenarios pipeline even flipped distributions! Delivery harmony both detail/confined enmpore airtight suitable with presence evidenced whilst checking intermediate datasets overview. \n\nThis aligns particularly toward sprawl with visually ringing appeal based on sampled dataset unsurprisingly synchronizes progressive observations promptly had exhaustive methods\n.\r\n//\r\n// Mapping digest against concise visual, salty enmurily variance prof observing/context-log heuristic; minimizes computational redundancy when akin implied assured refresh linear-bids stm varying removal as serach/fine algorithms for crafting significant key managerial/note assessments bridging summary visuals. Parented mapping dataset apt minority/visualizations asserts perpendicular scope validation matte dm proofquad better increments,...\n"
print(s)

\boxed{Histogram}

**Rationale:** 

The given summary statistics provide insights predominantly within the first six continuous variables: age, sex, BMI, BP, and four additional features (s1 to s6). Observing the mean, typical values, and variations captured by the standard deviation, we see that the variables cover both types that like normally distributed data as well as uniformly distributed ranges. Given this, a histogram would significantly benefit in visualizing and identifying the shape, spread, and central tendencies of these continuous values, noticing the identifiable peaks and spread (which can especially help with variables like age which can have skewness and multimodality in a larger dataset). Using plots as diversely as the others won't depict that specifically. Hence: 

1. Varied spread and outliers - histograms effectively show smooth distributions and variability, such as age-age who share nearly the same age or BMI following normal (equal spread).
2. The growing freq

## Wine

In [17]:
wine = load_wine()
df_wine = pd.DataFrame(wine.data, columns=wine.feature_names)
df_wine['target'] = [wine.target_names[t] for t in wine.target]

In [18]:
summary_statistics, sampled_rows = get_summary_stat_and_rows(df_wine)
prompt = get_prompt(summary_statistics,sampled_rows)
messages = [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

In [19]:
outputs = llm.generate(prompts, sampling_params, use_tqdm=False)
outputs[0].outputs[0].text #Box Plot

'Given the summary statistics and sample rows, a **Box Plot** is a highly insightful choice for this dataset.\n\n**Rationale for selecting Box Plot:**\n\n1. **Summary Statistics Analysis**:\n   - The dataset includes a wide variety of variables with predominantly continuous numerical data.\n   - The seven variables (`alcohol`, `malic_acid`, `ash`, `alcalinity_of_ash`, `magnesium`, `total_phenols`, `flavanoids`, `nonflavanoid_phenols`, `proanthocyanins`, `color_intensity`, `hue`, `od280/od315_of_diluted_wines`, `proline`) have a Kruskal-Wallis H test p-value significantly less than 0.05, indicating there are noticeable differences across categories in the target variable `target`.\n   - The box plots are well suited for visualizing numerical data comparisons and distribution which would help in understanding how wine **quality classification (`target`) impacts and differs among different variables**.\n\n2. **Quantitative Nature of Data**:\n   - The data shows substantial variance, both 

In [3]:
s = 'Given the summary statistics and sample rows, a **Box Plot** is a highly insightful choice for this dataset.\n\n**Rationale for selecting Box Plot:**\n\n1. **Summary Statistics Analysis**:\n   - The dataset includes a wide variety of variables with predominantly continuous numerical data.\n   - The seven variables (`alcohol`, `malic_acid`, `ash`, `alcalinity_of_ash`, `magnesium`, `total_phenols`, `flavanoids`, `nonflavanoid_phenols`, `proanthocyanins`, `color_intensity`, `hue`, `od280/od315_of_diluted_wines`, `proline`) have a Kruskal-Wallis H test p-value significantly less than 0.05, indicating there are noticeable differences across categories in the target variable `target`.\n   - The box plots are well suited for visualizing numerical data comparisons and distribution which would help in understanding how wine **quality classification (`target`) impacts and differs among different variables**.\n\n2. **Quantitative Nature of Data**:\n   - The data shows substantial variance, both within and between the total grams of each component like `alcohol`, `magnesium`, etc.\n   - Box plots allow quick comparison of the distribution of these components across the different classes of the `target`.\n\n3. **Readability and Concentrate on Key Values**:\n   - Shows the median (`mean`), quartiles, and outliers easily, making it useful for visualizing the spread and central tendency of continuous variables distinctly in a compelling way.\n   - Particularly benefits from the wide range of variables; each box indicating the 25th, 50th (median), and 75th Percentiles, primarily comparing different quantities and densities of juices easily interpretable.\n\n**Visual Insights**:\n- It will highlight changes in `alcohol`, `magnesium`, and others being recommended or unwanted for various classifications.\n- Stack format utility to show cumulative trend seen visually and how interdependent yield understandable dynamic adequately.\n\nConsidering these aspects, **12 alcoholic data distribution highlight via box plot** expertly captures thorough, user-friendly visualization necessary:\n\n\n*The admirable choice context form follow JSON completion requirement keeps fast due the stack specific plot nature detailed preliminary further comprehending.*'
print(s)

Given the summary statistics and sample rows, a **Box Plot** is a highly insightful choice for this dataset.

**Rationale for selecting Box Plot:**

1. **Summary Statistics Analysis**:
   - The dataset includes a wide variety of variables with predominantly continuous numerical data.
   - The seven variables (`alcohol`, `malic_acid`, `ash`, `alcalinity_of_ash`, `magnesium`, `total_phenols`, `flavanoids`, `nonflavanoid_phenols`, `proanthocyanins`, `color_intensity`, `hue`, `od280/od315_of_diluted_wines`, `proline`) have a Kruskal-Wallis H test p-value significantly less than 0.05, indicating there are noticeable differences across categories in the target variable `target`.
   - The box plots are well suited for visualizing numerical data comparisons and distribution which would help in understanding how wine **quality classification (`target`) impacts and differs among different variables**.

2. **Quantitative Nature of Data**:
   - The data shows substantial variance, both within and 

## Breast cancer

In [20]:
breast_cancer = load_breast_cancer()
df_breast_cancer = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
df_breast_cancer['target'] = [breast_cancer.target_names[t] for t in breast_cancer.target]

In [21]:
summary_statistics, sampled_rows = get_summary_stat_and_rows(df_breast_cancer)

In [22]:
prompt = get_prompt(summary_statistics,sampled_rows)

In [23]:
messages = [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

In [24]:
outputs = llm.generate(prompts, sampling_params, use_tqdm=False)
outputs[0].outputs[0].text #pair plot (scatterplot matrix)

'Given the summary statistics and the ten sample rows, I see a repeating pattern in numerical values, and there appear to be alphabetic columns, which might represent categorical data like target labels. To choose a suitable data visualization, we need to plot this information and show relationships or distributions efficiently.\n\nHere are the options we have:\n\n- **Histogram**: Good at showing frequencies across a numerical range.\n- **Box plot**: Good for displaying summary statistics and spotting outliers.\n- **Scatter plot**: Illustrates relationships or correlations between one numerical and one categorical variable (X vs Y, displaying different distributions or clusters).\n- **Bar chart**: Commonly used for counting or comparing discrete categories.\n- **Line chart**: Best for showing trends over time.\n- **Heatmap (correlation matrix)**: Effective for presenting the strength and direction of relationships between multiple variables.\n- **Pair plot (scatterplot matrix)**: Usefu

In [4]:
s = 'Given the summary statistics and the ten sample rows, I see a repeating pattern in numerical values, and there appear to be alphabetic columns, which might represent categorical data like target labels. To choose a suitable data visualization, we need to plot this information and show relationships or distributions efficiently.\n\nHere are the options we have:\n\n- **Histogram**: Good at showing frequencies across a numerical range.\n- **Box plot**: Good for displaying summary statistics and spotting outliers.\n- **Scatter plot**: Illustrates relationships or correlations between one numerical and one categorical variable (X vs Y, displaying different distributions or clusters).\n- **Bar chart**: Commonly used for counting or comparing discrete categories.\n- **Line chart**: Best for showing trends over time.\n- **Heatmap (correlation matrix)**: Effective for presenting the strength and direction of relationships between multiple variables.\n- **Pair plot (scatterplot matrix)**: Useful for comparing several pairs of numerical variables, often efficient in spotting trends or correlations.\n- **Violin plot**: Combines aspects of box plots and density plots and can also visualize the underlying distribution.\n\nTo decide on the optimal visualization:\n\n### Summary Statistics Highlights:\n- We have mostly numerical values which show means, standard deviations, minimums, and maximums.\n\n### Sample Rows Highlights:\n- Values consistently spread across new rows with great numeric variables ranging and including categorical outputs.\n\nGiven these, the most appropriate canvas to explore critical observations with variations and forming rich par timber understanding pairwise displaying drill include…\n\n**pair plot (scatterplot matrix)** - allows us to capture patterns, separations, pairs multi-informative and insightful relationships while adequately utilizing both alphabetic and numerics that come about when showed simultaneously.\n \n Below "original" reliability, with scattered sampling across varying distributions;\n\nThe heuristic caters both against many distributional out mean ranges, between providing rectangular correlation overlaps or vehicle inherent.\n\n\\boxed{pair plot (scatterplot matrix)}\n'
print(s)

Given the summary statistics and the ten sample rows, I see a repeating pattern in numerical values, and there appear to be alphabetic columns, which might represent categorical data like target labels. To choose a suitable data visualization, we need to plot this information and show relationships or distributions efficiently.

Here are the options we have:

- **Histogram**: Good at showing frequencies across a numerical range.
- **Box plot**: Good for displaying summary statistics and spotting outliers.
- **Scatter plot**: Illustrates relationships or correlations between one numerical and one categorical variable (X vs Y, displaying different distributions or clusters).
- **Bar chart**: Commonly used for counting or comparing discrete categories.
- **Line chart**: Best for showing trends over time.
- **Heatmap (correlation matrix)**: Effective for presenting the strength and direction of relationships between multiple variables.
- **Pair plot (scatterplot matrix)**: Useful for compa